### Ej 1

#### 1.1: Corpus

Comenzamos exportando los datos del archivo .tgz

In [1]:
import tarfile

In [2]:
tar = tarfile.open('ap.tgz')

In [3]:
tar.extractall()

Sobre el archivo .txt que contiene los datos de AP en xml aplicamos un parser manual para separar los documentos

In [4]:
docno = []
corpus = []
toggle = 0
with open('ap/ap.txt','r') as f:
    for line in f:
        if line.startswith('<DOCNO>'):
            line_new = line.replace('<DOCNO> ','')
            line_new = line_new.replace(' </DOCNO>\n','')
            docno.append(line_new)
        if toggle:
            corpus.append(line)
            toggle = 0
        if line.startswith('<TEXT>'):
            toggle = 1        
docs = dict(zip(docno, corpus))        

Otra forma sería usar el parser del ElementTree, pero para usarlo correctamente resulta necesario reemplazar todos los caracteres '&' por '&amp;' y agregar un tag que delimite el comienzo y final del archivo

In [6]:
with open('ap/ap.txt','r') as f, open('ap/ap_cleaned.txt','w') as f_new:
    f_new.write('<ROOT>\n')
    for line in f:
        new_line = line.replace('&','&amp;')
        f_new.write(new_line)
    f_new.write('</ROOT>\n')
f_new.close()

Ahora aplicamos el ElementTree

In [7]:
import xml.etree.ElementTree as ET

In [8]:
ap = ET.parse('ap/ap_cleaned.txt')
ap

In [9]:
ap_root = ap.getroot()

In [10]:
ap_dict = dict([
        (doc[0].text.strip(), doc[1].text)
        for doc in ap_root
    ])

Y observamos que de las 2 maneras separamos el xml en la misma cantidad de documentos

In [11]:
len(ap_dict)

2250

In [12]:
len(docs)

2250

#### 1.2: Tamaño vocabulario

In [13]:
import nltk

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HB\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Pruebo distintos tokenizadores

    Esto en ppio es solo para evaluar el proceso, pero no sería para presentar, así que una vez terminado se podría borrar. Igual conviene 1ro ver si se puede aplicar algo de esto para los ejercicios 1 y 2, que requieren tokenizar, eliminar puntuacion y esas cosas.

In [33]:
a = []
for text in ap_dict.values():
    a.append(text)

In [48]:
a[1]

"\n The Bechtel Group Inc. offered in 1985 to sell oil to Israel at a discount of at least $650 million for 10 years if it promised not to bomb a proposed Iraqi pipeline, a Foreign Ministry official said Wednesday. But then-Prime Minister Shimon Peres said the offer from Bruce Rappaport, a partner in the San Francisco-based construction and engineering company, was ``unimportant,'' the senior official told The Associated Press. Peres, now foreign minister, never discussed the offer with other government ministers, said the official, who spoke on condition of anonymity. The comments marked the first time Israel has acknowledged any offer was made for assurances not to bomb the planned $1 billion pipeline, which was to have run near Israel's border with Jordan. The pipeline was never built. In San Francisco, Tom Flynn, vice president for public relations for the Bechtel Group, said the company did not make any offer to Peres but that Rappaport, a Swiss financier, made it without Bechtel'

In [54]:
#b = []
#[b.append(text) for text in ap_dict.values()]
b = [text for text in ap_dict.values()]

In [55]:
b[1]

"\n The Bechtel Group Inc. offered in 1985 to sell oil to Israel at a discount of at least $650 million for 10 years if it promised not to bomb a proposed Iraqi pipeline, a Foreign Ministry official said Wednesday. But then-Prime Minister Shimon Peres said the offer from Bruce Rappaport, a partner in the San Francisco-based construction and engineering company, was ``unimportant,'' the senior official told The Associated Press. Peres, now foreign minister, never discussed the offer with other government ministers, said the official, who spoke on condition of anonymity. The comments marked the first time Israel has acknowledged any offer was made for assurances not to bomb the planned $1 billion pipeline, which was to have run near Israel's border with Jordan. The pipeline was never built. In San Francisco, Tom Flynn, vice president for public relations for the Bechtel Group, said the company did not make any offer to Peres but that Rappaport, a Swiss financier, made it without Bechtel'

In [158]:
parrafo = a[0]

In [159]:
words = parrafo.split(' ')

In [160]:
words

['\n',
 'A',
 '16-year-old',
 'student',
 'at',
 'a',
 'private',
 'Baptist',
 'school',
 'who',
 'allegedly',
 'killed',
 'one',
 'teacher',
 'and',
 'wounded',
 'another',
 'before',
 'firing',
 'into',
 'a',
 'filled',
 'classroom',
 'apparently',
 '``just',
 "snapped,''",
 'the',
 "school's",
 'pastor',
 'said.',
 '``I',
 "don't",
 'know',
 'how',
 'it',
 'could',
 'have',
 "happened,''",
 'said',
 'George',
 'Sweet,',
 'pastor',
 'of',
 'Atlantic',
 'Shores',
 'Baptist',
 'Church.',
 '``This',
 'is',
 'a',
 'good,',
 'Christian',
 'school.',
 'We',
 'pride',
 'ourselves',
 'on',
 'discipline.',
 'Our',
 'kids',
 'are',
 'good',
 "kids.''",
 'The',
 'Atlantic',
 'Shores',
 'Christian',
 'School',
 'sophomore',
 'was',
 'arrested',
 'and',
 'charged',
 'with',
 'first-degree',
 'murder,',
 'attempted',
 'murder,',
 'malicious',
 'assault',
 'and',
 'related',
 'felony',
 'charges',
 'for',
 'the',
 'Friday',
 'morning',
 'shooting.',
 'Police',
 'would',
 'not',
 'release',
 'the',


In [162]:
words2 = nltk.word_tokenize(parrafo)

In [233]:
words2

['A',
 '16-year-old',
 'student',
 'at',
 'a',
 'private',
 'Baptist',
 'school',
 'who',
 'allegedly',
 'killed',
 'one',
 'teacher',
 'and',
 'wounded',
 'another',
 'before',
 'firing',
 'into',
 'a',
 'filled',
 'classroom',
 'apparently',
 '``',
 'just',
 'snapped',
 ',',
 "''",
 'the',
 'school',
 "'s",
 'pastor',
 'said',
 '.',
 '``',
 'I',
 'do',
 "n't",
 'know',
 'how',
 'it',
 'could',
 'have',
 'happened',
 ',',
 "''",
 'said',
 'George',
 'Sweet',
 ',',
 'pastor',
 'of',
 'Atlantic',
 'Shores',
 'Baptist',
 'Church',
 '.',
 '``',
 'This',
 'is',
 'a',
 'good',
 ',',
 'Christian',
 'school',
 '.',
 'We',
 'pride',
 'ourselves',
 'on',
 'discipline',
 '.',
 'Our',
 'kids',
 'are',
 'good',
 'kids',
 '.',
 "''",
 'The',
 'Atlantic',
 'Shores',
 'Christian',
 'School',
 'sophomore',
 'was',
 'arrested',
 'and',
 'charged',
 'with',
 'first-degree',
 'murder',
 ',',
 'attempted',
 'murder',
 ',',
 'malicious',
 'assault',
 'and',
 'related',
 'felony',
 'charges',
 'for',
 'the'

In [163]:
len(words)

553

In [164]:
len(words2)

643

In [165]:
list(set(words)-set(words2))

["didn't",
 '',
 'us!',
 'grandmother,',
 'classmates.',
 '31,',
 'good,',
 '24,',
 'Marino,',
 'Elliott.',
 '``Once',
 'knapsack,',
 "don't",
 'juvenile,',
 'Williams,',
 "kids.''",
 'shooting.',
 'Firearms.',
 "Nicholas'",
 'Alcohol,',
 '``This',
 "it's",
 'that.',
 'discipline.',
 "happened,''",
 'there,',
 'classmate.',
 "snapped,''",
 '``just',
 'murder,',
 'adult,',
 'grade.',
 'meeting.',
 'students,',
 "school's",
 "us!''",
 "killed,''",
 'door.\n',
 'Sr.,',
 'them.',
 '``Jesus',
 'casings.',
 'request,',
 '``The',
 'fire,',
 'shot.',
 '40.',
 'shoulder.',
 'motive,',
 'Farley,',
 'said.',
 "ones.''",
 'gun,',
 'guns.',
 "guns,''",
 '\n',
 '``He',
 '``I',
 'grandfather,',
 'Allen,',
 'Matteson,',
 "youngster's",
 'Church.',
 'illness.',
 'said,',
 'teacher,',
 "grandmother's",
 "boy's",
 "youth's",
 'first.',
 'school.',
 'jammed,',
 'Sweet,']

In [166]:
match = [word for word in words if 'did' in word]
match

["didn't", "didn't"]

In [167]:
match2 = [word2 for word2 in words2 if 'did' in word2]
match2

['did', 'did']

In [168]:
match = [word for word in words if 'us' in word]
match

['``just',
 'malicious',
 'because',
 '``Jesus',
 'us!',
 "us!''",
 'serious',
 'Susan']

In [169]:
match2 = [word2 for word2 in words2 if 'us' in word2]
match2

['just', 'malicious', 'because', 'Jesus', 'us', 'us', 'serious', 'Susan']

O sea que en ppio sería mejor usar el tokenizador de nltk porque separa mejor los tokens que el split manual por espacios

In [170]:
words3 = nltk.wordpunct_tokenize(parrafo)

In [171]:
len(words3)

667

In [172]:
words3

['A',
 '16',
 '-',
 'year',
 '-',
 'old',
 'student',
 'at',
 'a',
 'private',
 'Baptist',
 'school',
 'who',
 'allegedly',
 'killed',
 'one',
 'teacher',
 'and',
 'wounded',
 'another',
 'before',
 'firing',
 'into',
 'a',
 'filled',
 'classroom',
 'apparently',
 '``',
 'just',
 'snapped',
 ",''",
 'the',
 'school',
 "'",
 's',
 'pastor',
 'said',
 '.',
 '``',
 'I',
 'don',
 "'",
 't',
 'know',
 'how',
 'it',
 'could',
 'have',
 'happened',
 ",''",
 'said',
 'George',
 'Sweet',
 ',',
 'pastor',
 'of',
 'Atlantic',
 'Shores',
 'Baptist',
 'Church',
 '.',
 '``',
 'This',
 'is',
 'a',
 'good',
 ',',
 'Christian',
 'school',
 '.',
 'We',
 'pride',
 'ourselves',
 'on',
 'discipline',
 '.',
 'Our',
 'kids',
 'are',
 'good',
 'kids',
 ".''",
 'The',
 'Atlantic',
 'Shores',
 'Christian',
 'School',
 'sophomore',
 'was',
 'arrested',
 'and',
 'charged',
 'with',
 'first',
 '-',
 'degree',
 'murder',
 ',',
 'attempted',
 'murder',
 ',',
 'malicious',
 'assault',
 'and',
 'related',
 'felony',
 

wordpunct_tokenize te separa tambien los simbolos de putuacion

In [101]:
list(set(words3)-set(words2))

["!''",
 'R',
 '22',
 '.,',
 ".''",
 'H',
 ",''",
 'degree',
 'Sr',
 's',
 'didn',
 'old',
 '37',
 '4',
 'don',
 'Ms',
 '16',
 't',
 '-',
 '32']

Otra forma sería eliminar los signos de puntuacion a mano

In [173]:
parrafo.translate(None, parrafo.punctuation)

AttributeError: 'str' object has no attribute 'punctuation'

No se porque no tienen el atributo, cuando supuestamente si lo tienen. Entonces uso un translator que me borre todas las puntuaciones obtenidas del paquete string. Pero eso me pega palabras. Quizas convendria reemplazar todas las puntuaciones por espacios.

In [106]:
import string

In [174]:
translator = parrafo.maketrans('','',string.punctuation)

In [175]:
parrafo.translate(translator)

'\n A 16yearold student at a private Baptist school who allegedly killed one teacher and wounded another before firing into a filled classroom apparently just snapped the schools pastor said I dont know how it could have happened said George Sweet pastor of Atlantic Shores Baptist Church This is a good Christian school We pride ourselves on discipline Our kids are good kids The Atlantic Shores Christian School sophomore was arrested and charged with firstdegree murder attempted murder malicious assault and related felony charges for the Friday morning shooting Police would not release the boys name because he is a juvenile but neighbors and relatives identified him as Nicholas Elliott Police said the student was tackled by a teacher and other students when his semiautomatic pistol jammed as he fired on the classroom as the students cowered on the floor crying Jesus save us God save us Friends and family said the boy apparently was troubled by his grandmothers death and the divorce of h

In [176]:
replace_punctuation = parrafo.maketrans(string.punctuation, ' '*len(string.punctuation))
parrafo_cleaned = parrafo.translate(replace_punctuation)
parrafo_cleaned

'\n A 16 year old student at a private Baptist school who allegedly killed one teacher and wounded another before firing into a filled classroom apparently   just snapped    the school s pastor said    I don t know how it could have happened    said George Sweet  pastor of Atlantic Shores Baptist Church    This is a good  Christian school  We pride ourselves on discipline  Our kids are good kids    The Atlantic Shores Christian School sophomore was arrested and charged with first degree murder  attempted murder  malicious assault and related felony charges for the Friday morning shooting  Police would not release the boy s name because he is a juvenile  but neighbors and relatives identified him as Nicholas Elliott  Police said the student was tackled by a teacher and other students when his semiautomatic pistol jammed as he fired on the classroom as the students cowered on the floor crying   Jesus save us  God save us    Friends and family said the boy apparently was troubled by his g

In [177]:
words4 = nltk.word_tokenize(parrafo_cleaned)

In [178]:
len(words4)

569

In [179]:
words4

['A',
 '16',
 'year',
 'old',
 'student',
 'at',
 'a',
 'private',
 'Baptist',
 'school',
 'who',
 'allegedly',
 'killed',
 'one',
 'teacher',
 'and',
 'wounded',
 'another',
 'before',
 'firing',
 'into',
 'a',
 'filled',
 'classroom',
 'apparently',
 'just',
 'snapped',
 'the',
 'school',
 's',
 'pastor',
 'said',
 'I',
 'don',
 't',
 'know',
 'how',
 'it',
 'could',
 'have',
 'happened',
 'said',
 'George',
 'Sweet',
 'pastor',
 'of',
 'Atlantic',
 'Shores',
 'Baptist',
 'Church',
 'This',
 'is',
 'a',
 'good',
 'Christian',
 'school',
 'We',
 'pride',
 'ourselves',
 'on',
 'discipline',
 'Our',
 'kids',
 'are',
 'good',
 'kids',
 'The',
 'Atlantic',
 'Shores',
 'Christian',
 'School',
 'sophomore',
 'was',
 'arrested',
 'and',
 'charged',
 'with',
 'first',
 'degree',
 'murder',
 'attempted',
 'murder',
 'malicious',
 'assault',
 'and',
 'related',
 'felony',
 'charges',
 'for',
 'the',
 'Friday',
 'morning',
 'shooting',
 'Police',
 'would',
 'not',
 'release',
 'the',
 'boy',
 's

Está mejor, pero el tema es que me está reemplazando también los apóstrofes por espacios y eso hace que palabras como don't se separen en don y t

Capaz conviene primero usar el word_tokenize del nltk, despues limpiarle las puntuaciones, volver a juntar el texto pero con espacios que separen la lista de tokens previa, y despues volver a tokenizar. Esto es porque hay algunas puntuaciones que el tokenizador usa para separar tokens.

In [234]:
wordss = nltk.word_tokenize(parrafo)

In [235]:
wordss

['A',
 '16-year-old',
 'student',
 'at',
 'a',
 'private',
 'Baptist',
 'school',
 'who',
 'allegedly',
 'killed',
 'one',
 'teacher',
 'and',
 'wounded',
 'another',
 'before',
 'firing',
 'into',
 'a',
 'filled',
 'classroom',
 'apparently',
 '``',
 'just',
 'snapped',
 ',',
 "''",
 'the',
 'school',
 "'s",
 'pastor',
 'said',
 '.',
 '``',
 'I',
 'do',
 "n't",
 'know',
 'how',
 'it',
 'could',
 'have',
 'happened',
 ',',
 "''",
 'said',
 'George',
 'Sweet',
 ',',
 'pastor',
 'of',
 'Atlantic',
 'Shores',
 'Baptist',
 'Church',
 '.',
 '``',
 'This',
 'is',
 'a',
 'good',
 ',',
 'Christian',
 'school',
 '.',
 'We',
 'pride',
 'ourselves',
 'on',
 'discipline',
 '.',
 'Our',
 'kids',
 'are',
 'good',
 'kids',
 '.',
 "''",
 'The',
 'Atlantic',
 'Shores',
 'Christian',
 'School',
 'sophomore',
 'was',
 'arrested',
 'and',
 'charged',
 'with',
 'first-degree',
 'murder',
 ',',
 'attempted',
 'murder',
 ',',
 'malicious',
 'assault',
 'and',
 'related',
 'felony',
 'charges',
 'for',
 'the'

In [236]:
for wordd in wordss:
    translatorr = wordd.maketrans('','',string.punctuation)
    print(wordd.translate(translatorr))

A
16yearold
student
at
a
private
Baptist
school
who
allegedly
killed
one
teacher
and
wounded
another
before
firing
into
a
filled
classroom
apparently

just
snapped


the
school
s
pastor
said


I
do
nt
know
how
it
could
have
happened


said
George
Sweet

pastor
of
Atlantic
Shores
Baptist
Church


This
is
a
good

Christian
school

We
pride
ourselves
on
discipline

Our
kids
are
good
kids


The
Atlantic
Shores
Christian
School
sophomore
was
arrested
and
charged
with
firstdegree
murder

attempted
murder

malicious
assault
and
related
felony
charges
for
the
Friday
morning
shooting

Police
would
not
release
the
boy
s
name
because
he
is
a
juvenile

but
neighbors
and
relatives
identified
him
as
Nicholas
Elliott

Police
said
the
student
was
tackled
by
a
teacher
and
other
students
when
his
semiautomatic
pistol
jammed
as
he
fired
on
the
classroom
as
the
students
cowered
on
the
floor
crying

Jesus
save
us

God
save
us


Friends
and
family
said
the
boy
apparently
was
troubled
by
his
grandmother
s
de

Esto separa bien el don't en do y n't (y le saca la puntuacion) pero no separa bien el 16-years-old. Esto es porque en algunos casos queremos que los signos de puntuacion separen tokens y en otros no. Lo que podemos hacer es usar de manera diferenciada algunos signos. Una opcion es definir que dont puede ser un token y entonces sacarle todas las puntuaciones menos el '-', aplicar el wordpunct_tokenize, y despues limpiarle los '-'. Esto queda bastante bien, salvo porque no separa don't como 2 tokens, pero si te separa lo demas.

In [253]:
all_punctuations = string.punctuation
selected_punctuations = re.sub(r'(\&|\-)', "", all_punctuations)

#punctuation_regex = re.compile('[%s]' % re.escape(selected_punctuations))
#punc_free = punctuation_regex.sub("",parrafo)
#print(punc_free)

replace_selected_punctuation = parrafo.maketrans(selected_punctuations, ' '*len(selected_punctuations))
punc_free = parrafo.translate(replace_selected_punctuation)
punc_free

'\n A 16-year-old student at a private Baptist school who allegedly killed one teacher and wounded another before firing into a filled classroom apparently   just snapped    the school s pastor said    I don t know how it could have happened    said George Sweet  pastor of Atlantic Shores Baptist Church    This is a good  Christian school  We pride ourselves on discipline  Our kids are good kids    The Atlantic Shores Christian School sophomore was arrested and charged with first-degree murder  attempted murder  malicious assault and related felony charges for the Friday morning shooting  Police would not release the boy s name because he is a juvenile  but neighbors and relatives identified him as Nicholas Elliott  Police said the student was tackled by a teacher and other students when his semiautomatic pistol jammed as he fired on the classroom as the students cowered on the floor crying   Jesus save us  God save us    Friends and family said the boy apparently was troubled by his g

In [254]:
wordsss = nltk.wordpunct_tokenize(punc_free)

In [256]:
' '.join(wordsss)

'A 16 - year - old student at a private Baptist school who allegedly killed one teacher and wounded another before firing into a filled classroom apparently just snapped the school s pastor said I don t know how it could have happened said George Sweet pastor of Atlantic Shores Baptist Church This is a good Christian school We pride ourselves on discipline Our kids are good kids The Atlantic Shores Christian School sophomore was arrested and charged with first - degree murder attempted murder malicious assault and related felony charges for the Friday morning shooting Police would not release the boy s name because he is a juvenile but neighbors and relatives identified him as Nicholas Elliott Police said the student was tackled by a teacher and other students when his semiautomatic pistol jammed as he fired on the classroom as the students cowered on the floor crying Jesus save us God save us Friends and family said the boy apparently was troubled by his grandmother s death and the di

O quizas conviene usar regular expressions

In [219]:
import re

In [215]:
palabras = re.split('\W+',parrafo)

In [217]:
len(palabras)

572

In [218]:
[palabra for palabra in palabras if 'did' in palabra]

['didn', 'didn']

También separa el didn't. Asi que el paquete 're' no agrega nada nuevo en ppio

Tokenizamos y pasamos a minúscula todos los textos.

In [18]:
def tokenize(string):
    'Returns list of tokens'
    # TODO: Cambiar split por: https://stackoverflow.com/questions/15547409/how-to-get-rid-of-punctuation-using-nltk-tokenizer
    words = string.split(' ')  # [Palabra]
    wnl = nltk.WordNetLemmatizer() 
    lemma_words = [wnl.lemmatize(word) for word in words]  # [Palabra]
    retoken = [w 
               for lw in lemma_words
               for w in nltk.word_tokenize(lw) ]
    return retoken

In [245]:
textos_tokenizados = [
    token
    for text in ap_dict.values()
    for token in tokenize(text)
]

In [246]:
print('Tamaño vocabulario: {}'.format(len(set(textos_tokenizados))))

Tamaño vocabulario: 46394


In [247]:
tokens_ordenados = sorted(textos_tokenizados)

In [248]:
from collections import Counter

In [249]:
c = Counter(tokens_ordenados)

In [250]:
c['doctor'] * 5

585

### Ej 2

Bajamos de [Project Gutenberg](http://www.gutenberg.org/) el libro de Darwin ON THE ORIGIN OF SPECIES 